In [ ]:
import pandas as pd
import numpy as np
import os
import math
pd.options.display.max_rows = 9999

In [ ]:
''' 오픈 소스 데이터에 누락된 값이 너무 많음 -> 새로 생성 후 계산  '''
# 파일 읽기 (탭 구분 txt → csv처럼 읽음)
dist_matrix = pd.read_csv(
    "E:/glass_git/AI-portfolio/order_batch_picking/data/Distance matrix/DistanceMatrix_36SubAisles.txt",
    sep="\t")

# null인 컬럼 삭제
cols_with_nulls = [col for col in dist_matrix.columns if dist_matrix[col].isnull().any()]
print(cols_with_nulls)
dist_matrix.drop(columns=cols_with_nulls,inplace=True)


#index 값인 0 컬럼을 기준으로 row 중복 제거
dist_matrix.drop_duplicates(subset=["0"], keep="first",inplace=True)
dist_matrix.index=dist_matrix['0']
dist_matrix.drop(columns=['0'],inplace=True)

# 중복 컬럼 삭제
cols_with_dot = dist_matrix.filter(regex=r'^\d+\.\d+$').columns.tolist()
print(cols_with_dot)
dist_matrix.drop(columns=cols_with_dot,inplace=True)

# dist_matrix.to_csv('E:/glass_git/AI-portfolio/order_batch_picking/data/Distance matrix/dist_matrix.csv',index=True)

In [ ]:
''' 
창고 레이아웃 설계
Rack : [1,24]
Bay : [1,120]
Tier= 0
Slot = [1,2880]

Aisle 간격 = 50
Rack 길이 = 1200
Bay 길이 = 10
'''
rack_num=list(range(1,25))
bay_num=list(range(1,121))

rack_bay_slot=[]
slot_num=1

for rack in rack_num:
    for bay in bay_num:
        for i in range(1,2):
            rack_bay_slot.append([rack,bay,slot_num])
            slot_num+=1
matrix=pd.DataFrame(rack_bay_slot,columns=['rack','bay','slot'])
matrix['aisle']=matrix['rack'].apply(lambda x: math.ceil(x/2))
matrix=matrix[['aisle','rack','bay','slot']]
matrix

,aisle,rack,bay,slot
0,1,1,1,1
1,1,1,2,2
2,1,1,3,3
3,1,1,4,4
4,1,1,5,5
...,...,...,...,...
2875,12,24,116,2876
2876,12,24,117,2877
2877,12,24,118,2878
2878,12,24,119,2879


In [ ]:
#pandas로 matrix 만들기 -> 속도가 너무 느림

max_bay_num=120
max_rack_num=24

total_dist=0
aisle_interval=50 #통로 너비
rack_length=1200 #rack 전체 길이
bay_length=10 #bay 세로 길이

def dist_stright(start, end):
    _,rack_s,bay_s=start
    _,rack_e,bay_e=end
    dist=abs(rack_s-rack_e)*aisle_interval+abs(bay_s-bay_e)*bay_length
    return dist

def dist_tail(start, end):
    aisle_s,rack_s,bay_s=start
    aisle_e,rack_e,bay_e=end
    #current_sate에서 tail로 이동
    move_tail=abs(max_bay_num-bay_s)*bay_length
    move_next_aile=abs(aisle_s-aisle_e)*aisle_interval
    #tail에서 item으로 이동
    move_item=abs(max_bay_num-bay_e)*bay_length
    dist=move_tail+move_next_aile+move_item
    return dist

def dist_head(start, end):
    aisle_s,rack_s,bay_s=start
    aisle_e,rack_e,bay_e=end
    #current_sate에서 head로 이동
    move_head=bay_s*bay_length
    move_next_aile=abs(aisle_s-aisle_e)*aisle_interval
    #head에서 item으로 이동
    move_item=bay_e*bay_length
    dist=move_head+move_next_aile+move_item
    return dist

def sort_slot(slot1,slot2):
    #move_stratage : tail, head, stright
    
    aisle1,rack1,bay1=slot1
    aisle2,rack2,bay2=slot2
   
    if aisle1==aisle2:
        if bay1>bay2:
            start=slot2
            end=slot1
           
        else:
            start=slot1
            end=slot2
        
        move_stratage='stright'
        return start,end,move_stratage
        
    else:
        if rack1>rack2:
            start=slot2
            end=slot1

        else:
            start=slot1
            end=slot2

        if start[2]>math.ceil(max_bay_num/2):
            move_stratage='tail'
        else:
            move_stratage='head'
            
        return start,end,move_stratage


def distance(slot1,slot2):

    start, end, move_stratage=sort_slot(slot1,slot2)
    if (slot1==slot2).all():
       dist=0
    else:
        if move_stratage=='stright':
            dist=dist_stright(start, end)
        elif move_stratage=='head':
            dist=dist_tail(start, end)
        else:
            dist=dist_head(start, end)
        
    return dist

slot_list=matrix.slot.unique()
matrix_=[]
for slot1_num in slot_list:
    row=[]
    for slot2_num in slot_list:
        slot1=matrix[matrix['slot']==slot1_num][['aisle','rack','bay']].values[0]
        slot2=matrix[matrix['slot']==slot2_num][['aisle','rack','bay']].values[0]

        dist=distance(slot1,slot2)
        row.append(dist)
    matrix_.append(row)

pd.DataFrame(matrix_)

In [ ]:
#numba로 distance matrix 생성

import numpy as np
import math
from numba import njit, prange

# ====== 상수 (Numba가 읽을 수 있게 전역으로 둠) ======
max_bay_num   = 120
aisle_interval = 50   # 통로 너비
bay_length     = 10   # bay 세로 길이
_mid_bay = (max_bay_num + 1) // 2  # ceil(max_bay_num/2)와 동일

# ====== 거리 함수 (Numba용) ======
@njit
def dist_stright_numba(rack_s, bay_s, rack_e, bay_e):
    return abs(rack_s - rack_e) * aisle_interval + abs(bay_s - bay_e) * bay_length

@njit
def dist_tail_numba(aisle_s, bay_s, aisle_e, bay_e):
    move_tail = abs(max_bay_num - bay_s) * bay_length
    move_next = abs(aisle_s - aisle_e) * aisle_interval
    move_item = abs(max_bay_num - bay_e) * bay_length
    return move_tail + move_next + move_item

@njit
def dist_head_numba(aisle_s, bay_s, aisle_e, bay_e):
    move_head = bay_s * bay_length
    move_next = abs(aisle_s - aisle_e) * aisle_interval
    move_item = bay_e * bay_length
    return move_head + move_next + move_item

@njit
def distance_numba(a1, r1, b1, a2, r2, b2):
    # 같은 aisle이면 straight
    if a1 == a2:
        return dist_stright_numba(r1, b1, r2, b2)

    # 다른 aisle이면 rack 작은 쪽을 start로 정렬
    if r1 > r2:
        sa, sr, sb = a2, r2, b2
        ea, er, eb = a1, r1, b1
    else:
        sa, sr, sb = a1, r1, b1
        ea, er, eb = a2, r2, b2

    # head/tail 결정 (시작 bay 기준)
    if sb > _mid_bay:
        return dist_tail_numba(sa, sb, ea, eb)
    else:
        return dist_head_numba(sa, sb, ea, eb)

@njit(parallel=True)
def build_dist_mat(A, R, B):
    N = A.shape[0]
    M = np.zeros((N, N), dtype=np.float64)
    for i in prange(N):
        ai, ri, bi = A[i], R[i], B[i]
        M[i, i] = 0.0
        for j in range(i + 1, N):
            d = distance_numba(ai, ri, bi, A[j], R[j], B[j])
            M[i, j] = d
            M[j, i] = d
    return M


slot_order =matrix.slot.unique()
df_coords = (
    matrix.set_index("slot")
          .loc[slot_order, ["aisle", "rack", "bay"]]
          .astype(np.int64)
)
coords = df_coords.to_numpy() 
A = coords[:, 0]
R = coords[:, 1]
B = coords[:, 2]


dist_mat = build_dist_mat(A, R, B)  


In [114]:
dist_mat

array([[   0.,   10.,   20., ..., 1740., 1750., 1760.],
       [  10.,    0.,   10., ..., 1750., 1760., 1770.],
       [  20.,   10.,    0., ..., 1760., 1770., 1780.],
       ...,
       [1740., 1750., 1760., ...,    0.,   10.,   20.],
       [1750., 1760., 1770., ...,   10.,    0.,   10.],
       [1760., 1770., 1780., ...,   20.,   10.,    0.]])

In [ ]:
dist_df = pd.DataFrame(dist_mat, index=slot_order, columns=slot_order)
dist_df.head()

,1,2,3,4,5,6,7,8,9,10,...,2871,2872,2873,2874,2875,2876,2877,2878,2879,2880
1,0.0,10.0,20.0,30.0,40.0,50.0,60.0,70.0,80.0,90.0,...,1670.0,1680.0,1690.0,1700.0,1710.0,1720.0,1730.0,1740.0,1750.0,1760.0
2,10.0,0.0,10.0,20.0,30.0,40.0,50.0,60.0,70.0,80.0,...,1680.0,1690.0,1700.0,1710.0,1720.0,1730.0,1740.0,1750.0,1760.0,1770.0
3,20.0,10.0,0.0,10.0,20.0,30.0,40.0,50.0,60.0,70.0,...,1690.0,1700.0,1710.0,1720.0,1730.0,1740.0,1750.0,1760.0,1770.0,1780.0
4,30.0,20.0,10.0,0.0,10.0,20.0,30.0,40.0,50.0,60.0,...,1700.0,1710.0,1720.0,1730.0,1740.0,1750.0,1760.0,1770.0,1780.0,1790.0
5,40.0,30.0,20.0,10.0,0.0,10.0,20.0,30.0,40.0,50.0,...,1710.0,1720.0,1730.0,1740.0,1750.0,1760.0,1770.0,1780.0,1790.0,1800.0


In [ ]:
# dist_df.to_pickle('E:/glass_git/AI-portfolio/order_batch_picking/data/Distance matrix/dist_matrix.pkl')

In [118]:
pd.read_pickle('E:/glass_git/AI-portfolio/order_batch_picking/data/Distance matrix/dist_matrix.pkl')

,1,2,3,4,5,6,7,8,9,10,...,2871,2872,2873,2874,2875,2876,2877,2878,2879,2880
1,0.0,10.0,20.0,30.0,40.0,50.0,60.0,70.0,80.0,90.0,...,1670.0,1680.0,1690.0,1700.0,1710.0,1720.0,1730.0,1740.0,1750.0,1760.0
2,10.0,0.0,10.0,20.0,30.0,40.0,50.0,60.0,70.0,80.0,...,1680.0,1690.0,1700.0,1710.0,1720.0,1730.0,1740.0,1750.0,1760.0,1770.0
3,20.0,10.0,0.0,10.0,20.0,30.0,40.0,50.0,60.0,70.0,...,1690.0,1700.0,1710.0,1720.0,1730.0,1740.0,1750.0,1760.0,1770.0,1780.0
4,30.0,20.0,10.0,0.0,10.0,20.0,30.0,40.0,50.0,60.0,...,1700.0,1710.0,1720.0,1730.0,1740.0,1750.0,1760.0,1770.0,1780.0,1790.0
5,40.0,30.0,20.0,10.0,0.0,10.0,20.0,30.0,40.0,50.0,...,1710.0,1720.0,1730.0,1740.0,1750.0,1760.0,1770.0,1780.0,1790.0,1800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2876,1720.0,1730.0,1740.0,1750.0,1760.0,1770.0,1780.0,1790.0,1800.0,1810.0,...,50.0,40.0,30.0,20.0,10.0,0.0,10.0,20.0,30.0,40.0
2877,1730.0,1740.0,1750.0,1760.0,1770.0,1780.0,1790.0,1800.0,1810.0,1820.0,...,60.0,50.0,40.0,30.0,20.0,10.0,0.0,10.0,20.0,30.0
2878,1740.0,1750.0,1760.0,1770.0,1780.0,1790.0,1800.0,1810.0,1820.0,1830.0,...,70.0,60.0,50.0,40.0,30.0,20.0,10.0,0.0,10.0,20.0
2879,1750.0,1760.0,1770.0,1780.0,1790.0,1800.0,1810.0,1820.0,1830.0,1840.0,...,80.0,70.0,60.0,50.0,40.0,30.0,20.0,10.0,0.0,10.0
